In [118]:
#%%bash
#pip install --upgrade pip

In [119]:
#%%bash
#pip install -r requirementsPyth.txt


In [100]:
"""
ATLAS Higgs Machine Learning Challenge 2014

#The algorithms higgsml_opendata_tmva.py was changed and adapted to work on a laptop and lxplus using a package scikit-learn instead TMVA/BDT.
"""
%matplotlib inline
from array import array
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix
import csv
import copy

#with open('/Users/okuchins/Olesya_work/HigsML2014/S_learn/atlas-higgs-challenge-2014-v2.csv','rb') as f:
#    reader = csv.reader(f)
#    your_list = map(tuple, reader)

#alldata = list(csv.reader(open('/Users/okuchins/Olesya_work/HigsML2014/S_learn/atlas-higgs-challenge-2014-v2.csv',"rb"), delimiter=','))
#list = map(tuple, reader)

In [101]:
#dataset
datafile="atlas-higgs-challenge-2014-v2.csv"
# load the CSV file as a numpy matrix
alldata = list(csv.reader(open(datafile,"r"), delimiter=','))

# first line is the list of variables, put it aside
header        = alldata.pop(0)

print("header", header)

header ['EventId', 'DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt', 'Weight', 'Label', 'KaggleSet', 'KaggleWeight']


In [102]:
# deal with some exception
iid=header.index("EventId")
ilabel=header.index("Label")
ikaggleset=header.index("KaggleSet")
ikaggleweight=header.index("KaggleWeight")
iweight=header.index("Weight") # original weight     
immc=header.index("DER_mass_MMC")
injet=header.index("PRI_jet_num")
exclude_index = [iid, ilabel, ikaggleset, ikaggleweight, iweight, immc, injet]
print("ilabel", ilabel) 

for entry in alldata:
    for i in range(len(entry)):
        if i in [iid,injet]:
            entry[i]=int(entry[i])
        elif i not in [ilabel,ikaggleset]:
            entry[i]=float(entry[i])

data_train = []
data_test = []



ilabel 32


In [103]:
sumselsig=0.
sumselbkg=0.
sumallsig=0.
sumallbkg=0.
sumsubsig=0.
sumsubbkg=0.
t=0
b=0
v=0
u=0

sumselkagglesig=0.
sumselkagglebkg=0.

for entry in alldata:
   
    weight=entry[iweight]
    kaggleweight=entry[ikaggleweight]    

    if entry[ilabel]=="s":
        entry[ilabel]=1
        sumallsig+=weight
    elif entry[ilabel]=="b":
        sumallbkg+=weight
        entry[ilabel]=0
    else:  
        entry[ilabel]=-999  

    if entry[ikaggleset]=="t":
        entry[ikaggleset]=0
        t+=1 
        data_train.append(entry)
    elif entry[ikaggleset]=="b":
        entry[ikaggleset]=10
        b+=1
        data_test.append(entry)
    elif entry[ikaggleset]=="v":
        entry[ikaggleset]=11
        v+=1
    elif entry[ikaggleset]=="u":
        entry[ikaggleset]=100
        u+=1      
    else:
        entry[ikaggleset]=-999

    if entry[ikaggleset]!=10:
        continue

    if entry[ilabel]==1:
        sumsubsig+=weight
    else:
        sumsubbkg+=weight  


print("Number of train is", (t))  
print("data_train", data_train[0:2])
print("Number of test is", (b))
print("data_test", data_test[0:2])    

Number of train is 250000
data_train [[100000, 138.47, 51.655, 97.827, 27.98, 0.91, 124.711, 2.666, 3.064, 41.928, 197.76, 1.582, 1.396, 0.2, 32.638, 1.017, 0.381, 51.626, 2.273, -2.414, 16.824, -0.277, 258.733, 2, 67.435, 2.15, 0.444, 46.062, 1.24, -2.475, 113.497, 0.00081448039868, 1, 0, 0.00265331133733], [100001, 160.937, 68.768, 103.235, 48.146, -999.0, -999.0, -999.0, 3.473, 2.078, 125.157, 0.879, 1.414, -999.0, 42.014, 2.039, -3.011, 36.918, 0.501, 0.103, 44.704, -1.916, 164.546, 1, 46.226, 0.725, 1.158, -999.0, -999.0, -999.0, 46.226, 0.681041906806, 0, 0, 2.23358448717]]
Number of test is 100000
data_test [[350000, -999.0, 79.589, 23.916, 3.036, -999.0, -999.0, -999.0, 0.903, 3.036, 56.018, 1.536, -1.404, -999.0, 22.088, -0.54, -0.609, 33.93, -0.504, -1.511, 48.509, 2.022, 98.556, 0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -0.0, 1.38090387249, 0, 10, 11.2466766407], [350008, 70.958, 66.329, 60.95, 0.758, -999.0, -999.0, -999.0, 2.37, 0.758, 66.573, 1.593, -1.38, -999.0

In [104]:
x_train = []
x_test = []
y_train = []
y_test = []

for row in data_train:
    newrow = copy.deepcopy(row)
    for el in sorted(exclude_index, reverse=True):
        del newrow[el]
    x_train.append(newrow)
    y_train.append(row[ilabel])

for row in data_test:
    newrow = copy.deepcopy(row)
    for el in sorted(exclude_index, reverse=True):
        del newrow[el]
    x_test.append(newrow)
    y_test.append(row[ilabel]) 
    


print("Length of X train is", (len(x_train)))  
print("x_train", x_train[0:1])
print("Length of Y train is", (len(y_train)))  
print("y_train", y_train[0:1])
print("Length of X test is", (len(x_test)))
print("x_test", x_test[0:1]) 
print("Length of Y test is", (len(y_test)))
print("y_test", y_test[0:1])    

Length of X train is 250000
x_train [[51.655, 97.827, 27.98, 0.91, 124.711, 2.666, 3.064, 41.928, 197.76, 1.582, 1.396, 0.2, 32.638, 1.017, 0.381, 51.626, 2.273, -2.414, 16.824, -0.277, 258.733, 67.435, 2.15, 0.444, 46.062, 1.24, -2.475, 113.497]]
Length of Y train is 250000
y_train [1]
Length of X test is 100000
x_test [[79.589, 23.916, 3.036, -999.0, -999.0, -999.0, 0.903, 3.036, 56.018, 1.536, -1.404, -999.0, 22.088, -0.54, -0.609, 33.93, -0.504, -1.511, 48.509, 2.022, 98.556, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -0.0]]
Length of Y test is 100000
y_test [0]


In [106]:
#alldata.head()

    
    # sum event weight passing the selection. Of course in real life the threshold should be optimised

#newlist=[] 
#signal = []
#for row in your_list[1:]:
    #if int(row[32]) == 0 && int(row[33]) == 0:
     #   backgr.append(int(row[23]))
    #else:
    #    signal.append(int(row[23]))

#print len(sinal)
#signal_tr = signal[:len(signal)*0.7]
#signal_ts = signal[len(signal)*0.7:]

    #newlist.append({'EventId':row[0],'Weight':row[31],'Label':row[32],'KaggleSet':row[33],'KaggleWeight':row[34],'Pri_jet_num':row[23]})

# where DecisionTrees, DecisionStumps and AdaBoost are
# compared to each other
#X, y = datasets.make_hastie_10_2(n_samples=12000, random_state=1)

# Train on the first 2000, test on the rest



#t_train, b_train = t[:2000], b[:2000]
#t_test, b_test = t[2000:], b[2000:]


# some shortcuts to select "signal" or "background"
# entries in the feautre vector
#signal = y_train > +0.1
#background = y_train < -0.1

#plt.scatter(X_train[signal,0], X_train[signal,1], c='red')
#plt.scatter(X_train[background,0], X_train[background,1], c='blue')
#plt.xlabel("Feature 0")
#plt.ylabel("Feature 1")

dt = DecisionTreeClassifier(max_depth=3,
                            min_samples_leaf=0.01*len(x_train[0]))
bdt = AdaBoostClassifier(dt,
                         algorithm='SAMME',
                         n_estimators=800,
                         learning_rate=0.5)

bdt.fit(x_train, y_train)
sk_y_predicted = bdt.predict(x_test)
print("sk_y_predicted", sk_y_predicted[:10])



sk_y_predicted [0 0 0 0 0 0 0 1 1 1]


In [117]:
print("Accuracy:", accuracy_score(y_test, sk_y_predicted))
print("Classification report:")
print(classification_report(y_test, sk_y_predicted,))
print("Confusion matrix:")
print(confusion_matrix(y_test, sk_y_predicted))


Accuracy: 0.79707
Classification report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.85     65975
           1       0.73      0.64      0.68     34025

   micro avg       0.80      0.80      0.80    100000
   macro avg       0.78      0.76      0.77    100000
weighted avg       0.79      0.80      0.79    100000

Confusion matrix:
[[57803  8172]
 [12121 21904]]
